In [1]:
from matplotlib.cm import get_cmap
from matplotlib.axes import Axes

In [2]:
import librosa
import librosa.display
import numpy as np
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import regularizers
from sklearn.utils import shuffle
import pandas as pd
import glob
import matplotlib.pyplot as plt
import scipy.io.wavfile
import sys
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
#from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from sklearn.model_selection import train_test_split
import os
import json

In [3]:
root = "D:/College Work/Internship/IIIT-A/Code/Audio_Trimmed_Videos/"

In [4]:
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(root):
    for file in f:
        files.append(file)

In [5]:
descriptors = pd.read_csv('Descriptors.csv', index_col=0)
descriptors.head()

,gender,gender_no,output,expression_index,video_name,start_time_in_sec,end_time_in_sec,start_exp,end_exp,text_file,kinect_start_frame,kinect_end_frame
1,F,3,An,1,927_0216_01.MP4,5.356700,9.481139,161,284,20170902_092359_00.txt,217,312
2,F,3,An,2,927_0216_01.MP4,9.883585,14.903150,296,447,20170902_092359_00.txt,322,466
3,F,3,An,3,927_0216_01.MP4,14.903150,19.427206,447,582,20170902_092359_00.txt,466,564
4,F,3,An,4,927_0216_01.MP4,19.427206,25.077029,582,752,20170902_092359_00.txt,564,699
5,F,3,An,5,927_0216_01.MP4,25.077029,29.981168,752,899,20170902_092359_00.txt,699,828


In [6]:
mylist = []
for aud in files:
    name = str(aud)[:-6] + '.MP4'
    emotion = descriptors.loc[descriptors['video_name']==name]['output'][:1].values[0]
    mylist.append(emotion)

In [7]:
feeling_list=[]
for item in mylist:
    if item=='Ne':
        feeling_list.append(0)
    elif item=='Sa':
        feeling_list.append(1)
    elif item=='Su':
        feeling_list.append(2)
    elif item=='Fe':
        feeling_list.append(3)
    elif item=='An':
        feeling_list.append(4)
    elif item=='Di':
        feeling_list.append(5)
    elif item=='Ha':
        feeling_list.append(6)

labels = pd.DataFrame(feeling_list)

In [8]:
df = pd.DataFrame(columns=['feature'])
bookmark = 0
for audio in files:
    X, sample_rate = librosa.load(root + audio, res_type='kaiser_fast',duration=None ,sr=None,offset=0.0)
    sample_rate = np.array(sample_rate)
    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=18), axis=0)
    feature = mfccs

    df.loc[bookmark] =[feature]
    bookmark=bookmark+1


In [9]:
df

,feature
0,"[-22.576344, -22.63391, -22.336678, -22.73294,..."
1,"[-16.56112, -15.109053, -14.493184, -13.463159..."
2,"[-27.23587, -26.376987, -24.637917, -25.186186..."
3,"[-25.14695, -23.783045, -24.60435, -24.942022,..."
4,"[-21.970781, -21.76817, -20.904003, -20.710114..."
...,...
310,"[-15.110401, -16.126104, -16.642399, -17.01920..."
311,"[-10.852483, -13.523667, -14.5473, -16.2716, -..."
312,"[-11.88979, -13.000632, -16.028648, -17.109066..."
313,"[-25.888529, -22.996035, -22.643534, -22.62645..."


In [10]:
idf3 = pd.DataFrame(df['feature'].values.tolist())

In [11]:
inewdf = pd.concat([idf3,labels], axis=1)

In [12]:
inewdf.index

RangeIndex(start=0, stop=315, step=1)

In [13]:
irnewdf = inewdf.rename(index=str, columns={0: "label"})

In [14]:
irnewdf.index

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '305', '306', '307', '308', '309', '310', '311', '312', '313', '314'],
      dtype='object', length=315)

In [15]:
irnewdf.head()

,label,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,label
0,-22.576344,-22.633909,-22.336678,-22.732941,-21.356634,-21.403225,-22.669485,-20.614010,-21.624844,-21.599819,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,-16.561119,-15.109053,-14.493184,-13.463159,-13.639797,-14.605306,-15.306802,-14.486601,-13.109497,-13.043911,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,-27.235870,-26.376987,-24.637917,-25.186186,-25.912806,-23.534084,-22.861095,-23.740396,-24.684076,-23.722967,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,-25.146950,-23.783045,-24.604349,-24.942022,-24.671974,-22.716316,-21.737007,-22.425535,-23.134193,-23.291124,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,-21.970781,-21.768169,-20.904003,-20.710114,-22.251064,-24.140152,-22.357712,-21.901991,-23.199156,-21.536785,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [16]:
irnewdf = irnewdf.fillna(0)

In [17]:
irnewdf

,label,1,2,3,4,5,6,7,8,9,...,1046,1047,1048,1049,1050,1051,1052,1053,1054,label
0,-22.576344,-22.633909,-22.336678,-22.732941,-21.356634,-21.403225,-22.669485,-20.614010,-21.624844,-21.599819,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,-16.561119,-15.109053,-14.493184,-13.463159,-13.639797,-14.605306,-15.306802,-14.486601,-13.109497,-13.043911,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,-27.235870,-26.376987,-24.637917,-25.186186,-25.912806,-23.534084,-22.861095,-23.740396,-24.684076,-23.722967,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,-25.146950,-23.783045,-24.604349,-24.942022,-24.671974,-22.716316,-21.737007,-22.425535,-23.134193,-23.291124,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,-21.970781,-21.768169,-20.904003,-20.710114,-22.251064,-24.140152,-22.357712,-21.901991,-23.199156,-21.536785,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,-15.110401,-16.126104,-16.642399,-17.019207,-17.975000,-18.825453,-19.487116,-19.043617,-20.585655,-22.298397,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
311,-10.852483,-13.523667,-14.547300,-16.271601,-16.924763,-17.488625,-18.852144,-20.408836,-20.884806,-21.933760,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
312,-11.889790,-13.000632,-16.028648,-17.109066,-15.882111,-15.968800,-16.425117,-18.963812,-19.968313,-17.888788,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
313,-25.888529,-22.996035,-22.643534,-22.626450,-21.710609,-22.616358,-21.866781,-19.947834,-19.194817,-20.434992,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6


In [18]:
irnewdf.size

332640

In [19]:
trainfeatures = irnewdf.iloc[:, :-1]
trainlabel = irnewdf.iloc[:,-1:]

x_train = np.array(trainfeatures)
y_train = np.array(trainlabel)
y_train = np_utils.to_categorical(y_train)

YY=y_train
XX =np.expand_dims(x_train, axis=2)

In [20]:
print(XX.shape, YY.shape)

(315, 1055, 1) (315, 7)


In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(XX,YY, test_size = 0.15)

In [22]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((267, 1055, 1), (48, 1055, 1), (267, 7), (48, 7))

In [23]:
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential

In [24]:
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], Y_train.shape[1]

In [28]:
model = Sequential()

model.add(Conv1D(64, 5,padding='same',
                 input_shape=(n_timesteps, n_features), kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(128, 5,padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(MaxPooling1D(pool_size=(4)))
model.add(Conv1D(256, 5,padding='same', kernel_regularizer=regularizers.l1_l2(l1=0.1, l2=0.1)))
model.add(Activation('relu'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(7))
model.add(Activation('softmax'))
opt = tf.keras.optimizers.RMSprop(learning_rate=0.00005, rho=0.9, epsilon=1e-07, decay=0.0)

In [29]:
verbose = 1
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [30]:
cnnhistory=model.fit(X_train, Y_train, batch_size=16, epochs=100, verbose=verbose)
_, accuracy = model.evaluate(X_test, Y_test, verbose=verbose)

Epoch 1/50
17/17 [==============================] - 1s 9ms/step - loss: 4.0661 - accuracy: 0.1760
Epoch 2/50
17/17 [==============================] - 0s 9ms/step - loss: 1.8144 - accuracy: 0.2734
Epoch 3/50
17/17 [==============================] - 0s 10ms/step - loss: 1.6921 - accuracy: 0.3034
Epoch 4/50
17/17 [==============================] - 0s 10ms/step - loss: 1.6011 - accuracy: 0.3371
Epoch 5/50
17/17 [==============================] - 0s 10ms/step - loss: 1.5128 - accuracy: 0.3596
Epoch 6/50
17/17 [==============================] - 0s 9ms/step - loss: 1.4010 - accuracy: 0.4382
Epoch 7/50
17/17 [==============================] - 0s 10ms/step - loss: 1.2697 - accuracy: 0.4569
Epoch 8/50
17/17 [==============================] - 0s 10ms/step - loss: 1.2144 - accuracy: 0.4794
Epoch 9/50
17/17 [==============================] - 0s 10ms/step - loss: 1.1295 - accuracy: 0.5094
Epoch 10/50
17/17 [==============================] - 0s 10ms/step - loss: 1.0576 - accuracy: 0.5468
Epoch 11/50
